In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
import numpy as np
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

22/09/14 12:05:18 WARN Utils: Your hostname, hexiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.179.36 instead (on interface en0)
22/09/14 12:05:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 12:05:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Read the given datasets

In [2]:
ori_transaction1 = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot').sort('order_datetime')
ori_transaction2 = spark.read.parquet('../data/tables/transactions_20210828_20220227_snapshot').sort('order_datetime')
ori_transaction3 = spark.read.parquet('../data/tables/transactions_20220228_20220828_snapshot').sort('order_datetime')

#read the parquet ori_transaction
ori_transaction = ori_transaction1.union(ori_transaction2)
ori_transaction = ori_transaction.union(ori_transaction3)
ori_transaction.groupby('merchant_abn').count().count()
ori_transaction.count()

14195505

In [32]:
ori_transaction

user_id,merchant_abn,dollar_value,order_datetime,whether_bigorder
714,10023283211,279.6185131488391,2021-02-28,0
5568,10023283211,110.07525568456784,2021-02-28,0
10285,10023283211,311.8728694054186,2021-02-28,0
23597,10023283211,189.25152107635006,2021-03-01,0
10144,10023283211,195.74454412745357,2021-03-01,0
22956,10023283211,230.81574259403658,2021-03-02,0
23856,10023283211,258.59035309253215,2021-03-02,0
10581,10023283211,211.8864214875864,2021-03-02,0
14842,10023283211,167.77961318216884,2021-03-02,0
13780,10023283211,175.9890781564819,2021-03-03,0


In [4]:
#delete outlier
wind = Window.partitionBy('merchant_abn')

q1= F.expr('percentile_approx(dollar_value, 0.25)')
q3= F.expr('percentile_approx(dollar_value, 0.75)')

testq = ori_transaction.withColumn('q1', q1.over(wind))
testq = testq.withColumn('q3', q3.over(wind))

testq = testq.withColumn('IQR', testq['q3']-testq['q1'])

In [6]:
dele_out = testq.where((testq["dollar_value"] <= testq["q1"]+1.5*testq["IQR"]) & (testq["dollar_value"] >= testq["q1"]-1.5*testq["IQR"]))

In [28]:
ori_transaction = dele_out.drop('q1','q3','IQR','order_id')

In [31]:
ori_transaction = ori_transaction.withColumn(
    "whether_bigorder",
    F.when(F.col('dollar_value')>=10000, 1).otherwise(0))
ori_transaction.filter(ori_transaction.whether_bigorder == 1)
ori_transaction.write.parquet("../data/curated/ori_transaction.parquet")

In [10]:
tbl_merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')
tbl_merchants.count()

4026

In [12]:
consumer_detail = spark.read.parquet('../data/tables/consumer_user_details.parquet')

In [13]:
import pandas as pd
tbl_consumer = spark.read.option("delimiter", "|").option("header",True).csv("../data/tables/tbl_consumer.csv")

In [14]:
import pandas as pd
tbl_consumer = spark.read.option("delimiter", "|").option("header",True).csv("../data/tables/tbl_consumer.csv")

# Merchant datasets

In [15]:
from pyspark.sql.functions import round
grouped_transaction = ori_transaction.groupBy('merchant_abn').agg(F.sum('dollar_value').alias('Amount'), F.count('dollar_value').alias('Count')).sort('merchant_abn')
grouped_transaction.drop(F.col('order_id'))
grouped_transaction = grouped_transaction.withColumn('Monthly Amount', round(grouped_transaction['Amount']/12, 2))
#grouped_transaction = grouped_transaction.withColumn('Round Monthly Amount', round(grouped_transaction['Monthly Amount'], 2))
grouped_transaction = grouped_transaction.withColumn('Monthly Count', round(grouped_transaction['Count']/12, 2))
grouped_transaction

merchant_abn,Amount,Count,Monthly Amount,Monthly Count
10023283211,515705.29636577313,2879,42975.44,239.92
10142254217,67344.50791712999,2583,5612.04,215.25
10165489824,56180.473857030534,5,4681.71,0.42
10187291046,28350.96112235929,297,2362.58,24.75
10192359162,121996.71300314639,338,10166.39,28.17
10206519221,210045.46392525383,8253,17503.79,687.75
10255988167,218547.72542909696,714,18212.31,59.5
10264435225,406049.91683000245,4373,33837.49,364.42
10279061213,116125.93771237625,474,9677.16,39.5
10323485998,868620.6233827489,8952,72385.05,746.0


In [16]:
grouped_transaction_pd = grouped_transaction.toPandas()
tbl_merchants_pd = tbl_merchants.toPandas()
merchant = pd.merge(grouped_transaction_pd, tbl_merchants_pd)

In [17]:
merchant

,merchant_abn,Amount,Count,Monthly Amount,Monthly Count,name,tags
0,10023283211,5.157053e+05,2879,42975.44,239.92,Felis Limited,"((furniture, home furnishings and equipment sh..."
1,10142254217,6.734451e+04,2583,5612.04,215.25,Arcu Ac Orci Corporation,"([cable, satellite, and otHer pay television a..."
2,10165489824,5.618047e+04,5,4681.71,0.42,Nunc Sed Company,"([jewelry, watch, clock, and silverware shops]..."
3,10187291046,2.835096e+04,297,2362.58,24.75,Ultricies Dignissim Lacus Foundation,"([wAtch, clock, and jewelry repair shops], [b]..."
4,10192359162,1.219967e+05,338,10166.39,28.17,Enim Condimentum PC,"([music shops - musical instruments, pianos, a..."
...,...,...,...,...,...,...,...
4021,99938978285,3.029251e+05,14962,25243.76,1246.83,Elit Dictum Eu Ltd,"[(opticians, optical goods, and eyeglasses), (..."
4022,99974311662,2.477399e+04,111,2064.50,9.25,Mollis LLP,"((books, periodicals, and newspapers), (b), (t..."
4023,99976658299,2.528178e+06,20200,210681.47,1683.33,Sociosqu Corp.,"((shoe shops), (a), (take rate: 6.57))"
4024,99987905597,5.328183e+04,173,4440.15,14.42,Commodo Hendrerit LLC,"[[motor vehicle Supplies and new parts], [a], ..."


In [18]:
for i in range(int(merchant['tags'].count())):
    merchant['tags'].iloc[i] = merchant['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')

/var/folders/4w/p31d1t_x5pqbppcsm7p9kxt00000gn/T/ipykernel_31146/1970225334.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merchant['tags'].iloc[i] = merchant['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')


In [19]:
#merchant['tags'].str.split(pat, n=-1, expand=False)
# to split into multiple columns by delimiter
merchant_tags = merchant['tags'].str.split(')', expand=True)

In [20]:
for row in range(int(len(merchant_tags))):
    for col in range(3):
        merchant_tags.iloc[row,col] = merchant_tags.iloc[row,col].replace(r'((', r'').replace(r', (', r'').replace(r'take rate:', r'')
merchant_tags.rename(columns = {0 : 'Store type', 1 : 'Revenue levels', 2 : 'Take rate'}, inplace = True)
merchant_tags = merchant_tags[['Store type', 'Revenue levels', 'Take rate']]

In [21]:
import pyspark.sql.functions as f
merchant[['Store type', 'Revenue levels', 'Take rate']] = merchant_tags[['Store type', 'Revenue levels', 'Take rate']]
#merchant.drop(columns=['tags'])
merchant['Store type'] = merchant['Store type'].str.lower()

for i in range(len(merchant)):
    merchant['Store type'][i] = ' '.join(merchant['Store type'][i].split())

#merchant
merchant.to_parquet("../data/curated/merchant.parquet")

/var/folders/4w/p31d1t_x5pqbppcsm7p9kxt00000gn/T/ipykernel_31146/203541969.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merchant['Store type'][i] = ' '.join(merchant['Store type'][i].split())


# Consumer datasets

In [22]:
consumer_detail_pd = consumer_detail.toPandas()
tbl_consumer_pd = tbl_consumer.toPandas()
tbl_consumer_pd['consumer_id']=tbl_consumer_pd['consumer_id'].astype(int)
consumer = pd.merge(tbl_consumer_pd, consumer_detail_pd)
consumer

,name,address,state,postcode,gender,consumer_id,user_id
0,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1195503,1
1,Mary Smith,3764 Amber Oval,NSW,2782,Female,179208,2
2,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,3
3,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female,154128,4
4,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female,712975,5
...,...,...,...,...,...,...,...
499994,Jessica Avila,508 Miranda Overpass Apt. 218,QLD,4400,Female,1385608,499995
499995,Steven Thornton,7913 Schwartz Mission Suite 483,VIC,3097,Undisclosed,1466964,499996
499996,Christy Smith,5681 Zachary Mountain Apt. 060,NSW,2756,Undisclosed,1253484,499997
499997,Donna Sutton,54140 Jacob Point,VIC,3989,Female,175005,499998


In [23]:
grouped_transaction = ori_transaction.groupBy('user_id').agg(F.avg('dollar_value').alias('avg'), F.sum('dollar_value').alias('sum'), F.count('dollar_value').alias('count')).sort('user_id')
grouped_transaction.drop(F.col('order_id'))
grouped_transaction_df = grouped_transaction.toPandas()
consumer_transaction = pd.merge(consumer, grouped_transaction_df)

consumer_transaction.to_parquet("../data/curated/consumer_transaction.parquet")

## Guess postcode for the merchant

In [24]:
merchant_abn_and_consumer_id = ori_transaction['merchant_abn', 'user_id']
#merchant_and_consumer_postcode['consumer postcode'] = c
merchant_abn_and_consumer_id
user_id_and_postcode = consumer[['postcode','user_id']]

user_id_and_postcode=spark.createDataFrame(user_id_and_postcode) 
#merchant_and_consumer_postcode = spark.merge(merchant_abn_and_consumer_id, user_id_and_postcode)
merchant_and_consumer_postcode = merchant_abn_and_consumer_id.join(user_id_and_postcode,['user_id'])
merchant_and_consumer_postcode = merchant_and_consumer_postcode['merchant_abn', 'postcode']
#merchant_and_consumer_postcode

#https://stackoverflow.com/questions/36654162/mode-of-grouped-data-in-pyspark
counts = merchant_and_consumer_postcode.groupBy(['merchant_abn', 'postcode']).count().alias('counts')
merchant_postcode = (counts
          .groupBy('merchant_abn')
          .agg(F.max(F.struct(F.col('count'),
                              F.col('postcode'))).alias('max'))
          .select(F.col('merchant_abn'), F.col('max.postcode'))
         )
merchant_postcode


22/09/14 12:07:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:07:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:07:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:07:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:07:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:07:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:07:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:07:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/09/14 12:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


merchant_abn,postcode
10023283211,5582
10142254217,6438
10165489824,6421
10187291046,5356
10192359162,9015
10206519221,5273
10255988167,3287
10264435225,6001
10279061213,6321
10323485998,4856


## Percentage of regular customer

In [25]:
ori_transaction_1 = ori_transaction.groupby('merchant_abn', 'user_id').count()
o_t = ori_transaction.groupby('merchant_abn').agg(F.count('user_id').alias('cnt'))
ori_con = ori_transaction_1.join(o_t, ori_transaction_1.merchant_abn == o_t.merchant_abn).drop(o_t.merchant_abn)
ori_con_drop = ori_con.filter(F.col("count") > 10)
ori_con_fix = ori_con_drop.groupby('merchant_abn').agg(F.count('user_id').alias('fix_cus_num'), F.avg('cnt').alias('total_cus_num'))
ori_con_fix_prob = ori_con_fix.withColumn("fix_cus_prob", ori_con_fix.fix_cus_num /ori_con_fix.total_cus_num)
ori_con_fix_prob

22/09/14 12:08:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:08:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


22/09/14 12:09:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:09:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:09:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/09/14 12:09:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


merchant_abn,fix_cus_num,total_cus_num,fix_cus_prob
32234779638,84,101568.0,8.270321361058601E-4
49891706470,10883,247281.0,0.04401065993748003
89726005175,4112,189258.0,0.02172695473903349
43186523025,2868,176115.0,0.016284813900008518
49505931725,1,59813.0,1.671877351077525E-5
60956456424,41,90705.0,4.520147731657571...
19933438190,2,57985.0,3.44916788824696E-5
45629217853,4698,195335.0,0.024050989326029643
48534649627,1,56805.0,1.760408414752222...
63290521567,1516,155454.0,0.009752081001453807


## Monthly new customer